In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from typing import Any, Dict, List, Tuple
from pathlib import Path
import yaml

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport # profile report を作る用
from matplotlib_venn import venn2 # venn図を作成する用
from tqdm import tqdm
from contextlib import contextmanager
from time import time
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import lightgbm as lgb

from mykaggle.metric.mse import rmsle
from mykaggle.util.ml_logger import MLLogger
from mykaggle.lib.lgbm_util import compute_importances, save_importances
from mykaggle.lib.pandas_util import change_column_name
from mykaggle.util.routine import fix_seed

sns.set_style('darkgrid')

In [2]:
settings = yaml.safe_load('''
name: '262_eda_scores'
competition: atmacup8
seed: 1019
training:
    num_folds: 5
    num_rounds: 1000
    early_stopping_rounds: 100
    verbose_eval: 20
lgbm_params:
    objective: binary
    learning_rate: 0.05
    max_depth: -1
    num_leaves: 31
    colsample_bytree: .7
    metric: "None"
feature:
    name_bow_pca_dim: 10
    name_bow_word_th1: 5
    name_bow_word_th2: 3
    name_bow_th1_upper: 130
    name_bow_th2_upper: 1000
''')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
fix_seed(settings['seed'])
pd.set_option('display.max_rows', 500)

In [4]:
datadir = Path('../data/')
ckptdir = Path('../ckpt/') / settings['name']
if not ckptdir.exists():
    ckptdir.mkdir()

In [5]:
df_train = pd.read_csv(datadir / 'id_train.csv')
df_test = pd.read_csv(datadir / 'id_test_unknown.csv')
df_submission = pd.read_csv(datadir / 'atmaCup8_sample-submission.csv')
df_train.shape, df_test.shape

((8359, 17), (8360, 12))

In [6]:
df_train.head().T

,0,1,2,3,4
Name,LEGO Batman: The Videogame,LEGO Indiana Jones: The Original Adventures,LEGO Batman: The Videogame,Combat,LEGO Harry Potter: Years 5-7
Platform,Wii,Wii,PSP,2600,Wii
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Warner Bros. Interactive Entertainment,LucasArts,Warner Bros. Interactive Entertainment,Atari,Warner Bros. Interactive Entertainment
NA_Sales,180,151,56,117,69
EU_Sales,97,61,44,7,42
JP_Sales,0,0,0,0,0
Other_Sales,28,21,27,1,12
Global_Sales,306,234,128,125,124


In [7]:
df_test.head().T

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4
Name,Hitman 2: Silent Assassin,Legacy of Kain: Soul Reaver,Metal Gear Solid 2: Substance,Silent Hill: Homecoming,Silent Hill: Homecoming
Platform,XB,PS,XB,X360,PS3
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Eidos Interactive,Eidos Interactive,Konami Digital Entertainment,Konami Digital Entertainment,Konami Digital Entertainment
Critic_Score,84,91,87,70,71
Critic_Count,23,17,28,54,41
User_Score,8,9,8.5,6.9,6.9
User_Count,19,132,39,180,143
Developer,Io Interactive,Crystal Dynamics,KCEJ,Double Helix Games,Double Helix Games


In [8]:
df_submission.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Global_Sales
0,63.371815
1,63.371815
2,63.371815
3,63.371815
4,63.371815


In [9]:
train = df_train.copy()
test = df_test.copy()
train['is_test'] = False
test['is_test'] = True
df_whole = pd.concat([train, test])

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
train_others = {
    'main': df_train.copy(),
    'another': df_test.copy()
}
test_others = {
    'main': df_test.copy(),
    'another': df_train.copy(),
}

# has sales のデータ作成

In [34]:
pred_sales = [
    '262_has_jp_sales',
    '263_has_na_sales',
    '264_has_eu_sales',
    '265_has_other_sales'
]

In [35]:
basedir = Path('../ckpt/')
dfs = []
for pred in pred_sales:
    df = pd.read_csv(basedir / pred / (pred + '.csv'))
    dfs.append(df)
    
df_sales = pd.concat(dfs, axis=1)
df_sales.head()

,has_jp_sales,has_na_sales,has_eu_sales,has_other_sales
0,0.030602,0.933737,0.914632,0.487030
1,0.497666,0.910432,0.955620,0.838640
2,0.103785,0.925141,0.910155,0.790746
3,0.325955,0.921452,0.943334,0.882707
4,0.545972,0.905374,0.931588,0.932657


In [36]:
df_test_sales = pd.concat([df_test, df_sales], axis=1)
df_test_sales.to_csv(datadir / 'region_prob_test_262.csv', index=False)
df_test_sales.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,has_jp_sales,has_na_sales,has_eu_sales,has_other_sales
0,Hitman 2: Silent Assassin,XB,NaN,Action,Eidos Interactive,84.0,23.0,8,19.0,Io Interactive,M,8359,0.030602,0.933737,0.914632,0.487030
1,Legacy of Kain: Soul Reaver,PS,NaN,Action,Eidos Interactive,91.0,17.0,9,132.0,Crystal Dynamics,T,8360,0.497666,0.910432,0.955620,0.838640
2,Metal Gear Solid 2: Substance,XB,NaN,Action,Konami Digital Entertainment,87.0,28.0,8.5,39.0,KCEJ,M,8361,0.103785,0.925141,0.910155,0.790746
3,Silent Hill: Homecoming,X360,NaN,Action,Konami Digital Entertainment,70.0,54.0,6.9,180.0,Double Helix Games,M,8362,0.325955,0.921452,0.943334,0.882707
4,Silent Hill: Homecoming,PS3,NaN,Action,Konami Digital Entertainment,71.0,41.0,6.9,143.0,Double Helix Games,M,8363,0.545972,0.905374,0.931588,0.932657


In [37]:
basedir = Path('../ckpt/')
dfs = []
for pred in pred_sales:
    df = pd.read_csv(basedir / pred / ('oof_' + pred + '.csv'))
    dfs.append(df)
    
df_sales = pd.concat(dfs, axis=1)
df_sales.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,has_jp_sales,has_na_sales,has_eu_sales,has_other_sales
0,0.010878,0.733617,0.977188,0.843998
1,0.025258,0.983060,0.925142,0.948982
2,0.009962,0.729503,0.690904,0.662269
3,0.008375,0.788722,0.805032,0.248701
4,0.009346,0.733617,0.963826,0.796361


In [38]:
has_sales = ['has_jp_sales', 'has_na_sales', 'has_eu_sales', 'has_other_sales']
df_train_sales = df_train.copy()
for c in has_sales:
    df_train_sales.loc[:, c] = df_sales.loc[:, c]
df_train_sales.to_csv(datadir / 'region_prob_train_oof_262.csv', index=False)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
basedir = Path('../ckpt/')
dfs = []
for pred in pred_sales:
    df = pd.read_csv(basedir / pred / ('train_' + pred + '.csv'))
    dfs.append(df)
    
df_sales = pd.concat(dfs, axis=1)
df_sales.head()

,has_jp_sales,has_na_sales,has_eu_sales,has_other_sales
0,0.022929,0.935234,0.944154,0.951830
1,0.025009,0.931853,0.946925,0.923892
2,0.020638,0.931021,0.868304,0.911874
3,0.008871,0.890094,0.909186,0.272971
4,0.024498,0.934017,0.939700,0.942884


In [40]:
has_sales = ['has_jp_sales', 'has_na_sales', 'has_eu_sales', 'has_other_sales']
df_train_sales = df_train.copy()
for c in has_sales:
    df_train_sales.loc[:, c] = df_sales.loc[:, c]
df_train_sales.to_csv(datadir / 'region_prob_train_262.csv', index=False)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# User Score など

In [22]:
df_train['is_User_Score_tbd'] = df_train['User_Score'].apply(lambda x: x == 'tbd')
df_train.loc[df_train['User_Score'] == 'tbd', 'User_Score'] = np.nan
df_train['User_Score'] = df_train['User_Score'].astype(np.float32)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
df_train_score = df_train[~df_train['is_User_Score_tbd']]
df_train_score['User_Score'] = df_train_score['User_Score'].astype(float)

In [13]:
# rating system がはじまったのはいつから?
columns = [
    'User_Score', 'User_Count', 'Critic_Score', 'Critic_Count',
    'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'
]
df_train_score.groupby('Year_of_Release')[columns].mean()

,User_Score,User_Count,Critic_Score,Critic_Count,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Year_of_Release,,,,,,,,,
1980.0,NaN,NaN,NaN,NaN,194.250000,12.250000,0.000000,2.250000,208.750000
1981.0,NaN,NaN,NaN,NaN,68.352941,4.029412,0.000000,0.676471,73.264706
1982.0,NaN,NaN,NaN,NaN,94.565217,5.782609,0.000000,1.043478,101.347826
1983.0,NaN,NaN,NaN,NaN,42.500000,4.928571,57.857143,0.857143,105.928571
1984.0,NaN,NaN,NaN,NaN,361.888889,21.666667,115.111111,7.444444,506.222222
1985.0,NaN,NaN,NaN,NaN,414.125000,58.500000,152.875000,11.375000,636.875000
1986.0,NaN,NaN,NaN,NaN,84.454545,19.909091,80.181818,16.636364,201.000000
1987.0,NaN,NaN,NaN,NaN,62.181818,11.727273,78.727273,1.727273,154.727273
1988.0,2.200000,4572.000000,64.000000,75.000000,209.727273,59.545455,122.090909,8.909091,400.363636


In [14]:
# Platform ごとのスコアは?
columns = [
    'User_Score', 'User_Count', 'Critic_Score', 'Critic_Count',
    'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'
]
df_train_score.groupby('Platform')[columns].mean()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,User_Score,User_Count,Critic_Score,Critic_Count,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Platform,,,,,,,,,
2600,NaN,NaN,NaN,NaN,75.170455,4.556818,0.000000,0.750000,80.568182
3DO,NaN,NaN,NaN,NaN,0.000000,0.000000,2.000000,0.000000,2.000000
3DS,7.172641,128.198113,70.700000,37.950000,28.130952,21.317460,33.361111,4.567460,87.424603
DC,8.436364,82.000000,86.818182,17.181818,12.243902,3.560976,16.195122,0.585366,32.634146
DS,7.186054,42.625850,69.088968,27.241993,28.640854,17.750334,19.062750,5.173565,70.662216
GB,NaN,NaN,NaN,NaN,144.166667,60.012821,92.474359,10.269231,306.987179
GBA,7.658824,29.732026,69.815068,18.438356,31.950156,13.451713,12.937695,1.647975,60.052960
GC,7.691824,63.509434,69.857143,25.785714,39.444444,11.838384,9.813131,1.727273,62.919192
GEN,NaN,NaN,NaN,NaN,60.095238,18.380952,12.380952,3.000000,94.000000


In [ ]:
# sns.pairplot(df_train[columns])

In [ ]:
# データを Score あるなしで分けてみる

In [ ]:
df_train.groupby('Year_of_Release')['Name'].count()

In [ ]:
# from mykaggle.feature.score_nan import ScoreNaN
# sn_train = ScoreNaN(train=True)
# sn_test = ScoreNaN(train=False)
# df_f_train = sn_train(df_train[['id']].copy(), others=train_others, use_cache=False, save_cache=False)
# df_f_test = sn_test(df_test[['id']].copy(), others=test_others, use_cache=False, save_cache=False)
# df_f_train

In [15]:
us_na_tr = df_train['User_Score'].isna()
uc_na_tr = df_train['User_Count'].isna()
cs_na_tr = df_train['Critic_Score'].isna()
cc_na_tr = df_train['Critic_Count'].isna()
us_na_te = df_test['User_Score'].isna()
uc_na_te = df_test['User_Count'].isna()
cs_na_te = df_test['Critic_Score'].isna()
cc_na_te = df_test['Critic_Count'].isna()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
df_train.loc[us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr].shape, df_test.loc[us_na_te & uc_na_te & cs_na_te & cc_na_te].shape

((4126, 18), (3156, 12))

In [17]:
df_train.loc[us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr].shape, df_test.loc[us_na_te & uc_na_te & cs_na_te & cc_na_te].shape

((4126, 18), (3156, 12))

In [ ]:
df_no_score_train = df_train.loc[us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr]
df_no_score_test = df_test.loc[us_na_te & uc_na_te & cs_na_te & cc_na_te]
df_score_train = df_train.loc[~(us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr)]
df_score_test = df_test.loc[~(us_na_te & uc_na_te & cs_na_te & cc_na_te)]

In [20]:
df_train = pd.read_csv(datadir / 'id_train.csv')
df_test = pd.read_csv(datadir / 'id_test_unknown.csv')
us_na_tr = df_train['User_Score'].isna()
uc_na_tr = df_train['User_Count'].isna()
cs_na_tr = df_train['Critic_Score'].isna()
cc_na_tr = df_train['Critic_Count'].isna()
us_na_te = df_test['User_Score'].isna()
uc_na_te = df_test['User_Count'].isna()
cs_na_te = df_test['Critic_Score'].isna()
cc_na_te = df_test['Critic_Count'].isna()

df_no_score_train = df_train.loc[us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr]
df_no_score_test = df_test.loc[us_na_te & uc_na_te & cs_na_te & cc_na_te]
df_score_train = df_train.loc[~(us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr)]
df_score_test = df_test.loc[~(us_na_te & uc_na_te & cs_na_te & cc_na_te)]

# df_no_score_train.to_csv(datadir / 'id_train_no_score.csv', index=False)
# df_no_score_test.to_csv(datadir / 'id_test_no_score.csv', index=False)
# df_score_train.to_csv(datadir / 'id_train_score.csv', index=False)
# df_score_test.to_csv(datadir / 'id_test_score.csv', index=False)
df_no_score_train.shape, df_score_train.shape, df_no_score_test.shape, df_score_test.shape

((3510, 17), (4849, 17), (3156, 12), (5204, 12))

In [52]:
SALES_COLUMNS = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
df_train.groupby('Platform')[SALES_COLUMNS].agg(['mean'])

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
,mean,mean,mean,mean,mean
Platform,,,,,
2600,75.170455,4.556818,0.000000,0.750000,80.568182
3DO,0.000000,0.000000,2.000000,0.000000,2.000000
3DS,26.600000,20.018519,31.200000,4.296296,82.159259
DC,12.243902,3.560976,16.195122,0.585366,32.634146
DS,23.631103,13.197107,13.083183,3.971971,53.908680
GB,144.166667,60.012821,92.474359,10.269231,306.987179
GBA,27.777528,11.676404,9.478652,1.346067,50.370787
GC,34.020833,10.137500,8.095833,1.462500,53.812500


# 再度 User Score など

In [78]:
df_train.groupby(['Platform'])['User_Score', 'Critic_Score'].agg(['count', 'mean', 'std'])

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


User_Score                     Critic_Score                      
              count      mean       std        count       mean        std
Platform                                                                  
2600              0       NaN       NaN            0        NaN        NaN
3DO               0       NaN       NaN            0        NaN        NaN
3DS             106  7.172641  1.248201          103  70.349515  11.321253
DC               11  8.436363  1.023008           11  86.818182   5.192652
DS              294  7.186054  1.551365          408  65.644608  14.006445
GB                0       NaN       NaN            0        NaN        NaN
GBA             153  7.658823  1.454599          231  67.082251  14.126361
GC              159  7.691824  1.293458          190  68.205263  13.949834
GEN               0       NaN       NaN            0        NaN        NaN
GG                0       NaN       NaN            0        NaN        NaN
N64               0       NaN       NaN            0        NaN        NaN
NES               0       NaN       NaN            0        NaN        NaN
NG                0       NaN       NaN            0        NaN        NaN
PC              354  7.238418  1.464014          329  76.106383  11.204757
PCFX              0       NaN       NaN            0        NaN        NaN
PS               65  8.012308  1.101633           79  73.240506  15.434074
PS2             599  7.663773  1.310457          619  68.609047  12.522303
PS3             428  6.956776  1.315188          403  70.756824  13.718036
PS4             124  7.075807  1.138632          122  73.172131  11.432658
PSP             225  7.304000  1.267073          247  69.044534  12.026142
PSV              86  7.520930  1.124917           75  72.306667   9.666008
SAT               0       NaN       NaN            0        NaN        NaN
SCD               0       NaN       NaN            0        NaN        NaN
SNES              0       NaN       NaN            0        NaN        NaN
TG16              0       NaN       NaN            0        NaN        NaN
WS                0       NaN       NaN            0        NaN        NaN
Wii             267  6.935581  1.550760          287  63.651568  14.417011
WiiU             63  7.292064  1.308570           59  71.084746  14.597222
X360            442  6.786878  1.420919          426  68.600939  14.734686
XB              239  7.408787  1.681250          309  67.809061  14.715013
XOne             84  6.866667  1.204543           78  74.346154  12.665350

In [79]:
df_train = pd.read_csv(datadir / 'country_prob_train.csv')
df_test = pd.read_csv(datadir / 'country_prob_test_unknown.csv')
us_na_tr = df_train['User_Score'].isna()
uc_na_tr = df_train['User_Count'].isna()
cs_na_tr = df_train['Critic_Score'].isna()
cc_na_tr = df_train['Critic_Count'].isna()
us_na_te = df_test['User_Score'].isna()
uc_na_te = df_test['User_Count'].isna()
cs_na_te = df_test['Critic_Score'].isna()
cc_na_te = df_test['Critic_Count'].isna()

df_no_score_train = df_train.loc[us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr]
df_no_score_test = df_test.loc[us_na_te & uc_na_te & cs_na_te & cc_na_te]
df_score_train = df_train.loc[~(us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr)]
df_score_test = df_test.loc[~(us_na_te & uc_na_te & cs_na_te & cc_na_te)]

# df_no_score_train.to_csv(datadir / 'country_prob_train_no_score.csv', index=False)
# df_no_score_test.to_csv(datadir / 'country_prob_test_no_score.csv', index=False)
# df_score_train.to_csv(datadir / 'country_prob_train_score.csv', index=False)
# df_score_test.to_csv(datadir / 'country_prob_test_score.csv', index=False)

df_no_score_train.shape, df_score_train.shape, df_no_score_test.shape, df_score_test.shape

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


((3510, 21), (4849, 21), (3156, 16), (5204, 16))

In [80]:
np.sum(df_train.loc[~df_train['User_Score'].isna(), 'Critic_Score'].isna())

873